In [30]:
#import required dependencies

import csv
import random
import math
import os
import shutil
import h2o
from h2o.automl import H2OAutoML
import pandas as pd
import matplotlib.pyplot as plt

In [31]:
#Technology Stack: Python, H2O, SHAP, and Google Generative AI
# Filename: 1_generate_dga_data.py

def get_entropy(s):
    p, lns = {}, float(len(s))
    for c in s:
        p[c] = p.get(c, 0) + 1
    return -sum(count/lns * math.log(count/lns, 2) for count in p.values())

# Create sample data
header = ['domain', 'length', 'entropy', 'class']
data = []
# Legitimate domains
legit_domains = ['google', 'facebook', 'amazon', 'github', 'wikipedia', 'microsoft']
for _ in range(100):
    domain = random.choice(legit_domains) + ".com"
    data.append([domain, len(domain), get_entropy(domain), 'legit'])
# DGA domains
for _ in range(100):
    length = random.randint(15, 25)
    domain = ''.join(random.choice('abcdefghijklmnopqrstuvwxyz0123456789') for _ in range(length)) + ".com"
    data.append([domain, len(domain), get_entropy(domain), 'dga'])

with open('dga_dataset_train.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)

print("dga_dataset_train.csv created successfully.")

dga_dataset_train.csv created successfully.


In [32]:
#required downloads per the h2o.ai documents site
!pip install requests
!pip install tabulate
!pip install future

# Required for plotting:
!pip install matplotlib

In [11]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
   ---------------------------------------- 0.0/265.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/265.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/265.9 MB ? eta -:--:--
   ---------------------------------------- 0.1/265.9 MB 544.7 kB/s eta 0:08:09
   ---------------------------------------- 1.0/265.9 MB 7.0 MB/s eta 0:00:38
    --------------------------------------- 4.0/265.9 MB 23.0 MB/s eta 0:00:12
   - -------------------------------------- 7.1/265.9 MB 32.5 MB/s eta 0:00:08
   - -------------------------------------- 11.1/265.9 MB 72.6 MB/s eta 0:00:04
   -- ------------------------------------- 14.8/265.9 MB 81.8 MB/s eta 0:00:04
   -- ------------------------------------- 18.8/265.9 MB 81.8 MB/s eta 0:00:04
   --- ------------------------------------ 22.7/265.9 MB 81.8 MB/s eta 0:00:03
   ---- ----------------------------------- 26.7/265.9 MB 93.9 MB/s eta

In [33]:
# Filename: 2_run_automl.py
h2o.init()

train = h2o.import_file("dga_dataset_train.csv") #train 
x = ['length', 'entropy'] # Features as x input
y = "class"               # Target as variables
train[y] = train[y].asfactor()

aml = H2OAutoML(max_models=20, max_runtime_secs=120, seed=1) #instanciating automl
aml.train(x=x, y=y, training_frame=train)

print("H2O AutoML process complete.")
print("Leaderboard (Top 10):")
print(aml.leaderboard.as_data_frame().head(10))

# (Add this to the end of 2_run_automl.py)
# Get the best performing model from the leaderboard
best_model = aml.leader

# Adding output directory
os.makedirs("./models", exist_ok=True)

# Download the MOJO artifact.
mojo_path = best_model.download_mojo(path="./models/")

# Adding try, except in the event there are errors
mojo_path = None
try:
    # Many models support MOJO; some (depending on build/type) may not.
    # get_genmodel_jar=True is handy if you plan to score with Java.
    mojo_path = best_model.download_mojo(path="./models/")
    print(f"MOJO saved to: {mojo_path}")
except Exception as e:
    print(f"Leader MOJO not available ({e}). Trying next MOJO-capable model...")
    # Walk down the leaderboard and pick the first MOJO-capable model
    lb_df = aml.leaderboard.as_data_frame()
    for mid in lb_df['model_id'].tolist():
        try:
            m = h2o.get_model(mid)
            mojo_path = m.download_mojo(path="./models/")
            print(f"MOJO saved from fallback model {mid} to: {mojo_path}")
            break
        except Exception:
            continue
    if mojo_path is None:
        print("No MOJO-capable model found. You can still use the binary model.")

# Save the binary model (always works)
model_path = h2o.save_model(model=best_model, path="./models", force=True)
print(f"Binary model saved to: {model_path}")

# After AutoML finishes
custom_name = "best_dga_model"
base, ext = os.path.splitext(model_path)          # keep .zip/.bin/etc
new_path = os.path.join("./models", custom_name + ext)
shutil.move(model_path, new_path)
print(f"Renamed model path: {new_path}")

if mojo_path:
    print(f"Production-ready MOJO at: {mojo_path}")
else:
    print("MOJO not produced; use the binary model path above for Python scoring.")

# Clean shutdown (non-interactive)
h2o.shutdown(prompt=False)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM Temurin-11.0.26+4 (build 11.0.26+4, mixed mode)
  Starting server from C:\Users\vegiv\anaconda3\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\vegiv\AppData\Local\Temp\tmpfu3dxjbo
  JVM stdout: C:\Users\vegiv\AppData\Local\Temp\tmpfu3dxjbo\h2o_vegiv_started_from_python.out
  JVM stderr: C:\Users\vegiv\AppData\Local\Temp\tmpfu3dxjbo\h2o_vegiv_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,4 months and 24 days
H2O_cluster_name:,H2O_from_python_vegiv_nb030t
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.928 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
13:17:47.778: AutoML: XGBoost is not available; skipping it.


13:17:48.972: GBM_1_AutoML_1_20250821_131747 [GBM def_5] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for GBM model: GBM_1_AutoML_1_20250821_131747.  Details: ERRR on field: _min_rows: The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 160.0.
ERRR on field: _min_rows: The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 160.0.
ERRR on field: _min_rows: The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 160.0.
ERRR on field: _min_rows: The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 160.0.
ERRR on field: _min_rows: The d

C:\Users\vegiv\anaconda3\Lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


MOJO saved to: C:\Users\vegiv\models\StackedEnsemble_AllModels_1_AutoML_1_20250821_131747.zip
Binary model saved to: C:\Users\vegiv\models\StackedEnsemble_AllModels_1_AutoML_1_20250821_131747
Renamed model path: ./models\best_dga_model
Production-ready MOJO at: C:\Users\vegiv\models\StackedEnsemble_AllModels_1_AutoML_1_20250821_131747.zip
H2O session _sid_9f56 closed.


C:\Users\vegiv\AppData\Local\Temp\ipykernel_38864\3436138701.py:65: H2ODeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  h2o.shutdown(prompt=False)


Robustness: Handles the case where the leader lacks a MOJO by gracefully falling back.
Usability: Leaderboard prints cleanly; saved model keeps its extension.
Reliability: Ensures paths exist and shutdown won’t hang a batch job.

In [23]:
#SHAP (SHapley Additive exPlanations) can be installed using either pip or conda
!pip install shap
import shap

   ---------------------------------------- 0.0/544.4 kB ? eta -:--:--
   --- ------------------------------------ 41.0/544.4 kB 1.9 MB/s eta 0:00:01
   ---------------------------------------- 544.4/544.4 kB 8.5 MB/s eta 0:00:00


In [39]:
# Filename: 3_explain_model.py

h2o.init()
# Load the saved model (replace with your actual model ID from the leaderboard)
# model_id = "StackedEnsemble_AllModels_1_AutoML_1_..."
# best_model = h2o.get_model(model_id)

model_path = "./models/best_dga_model"
best_model = h2o.load_model(model_path)

# Load test data for explanation
test_df = pd.read_csv("dga_dataset_train.csv")
X_test = test_df[['length', 'entropy']]

def predict_wrapper(data): #acts as bridge between SHAP and h2o
    h2o_df = h2o.H2OFrame(pd.DataFrame(data, columns=X_test.columns))
    predictions = best_model.predict(h2o_df)
    return predictions.as_data_frame()['dga']

explainer = shap.KernelExplainer(predict_wrapper, X_test.head(50))
shap_values = explainer.shap_values(X_test.head(50))

print("Displaying SHAP Summary Plot (Global Explanation)...")
shap.summary_plot(shap_values, X_test.head(50), show=False)
plt.savefig("shap_summary.png")
plt.close()

print("Displaying SHAP Force Plot (Local Explanation for first instance)...")
shap.force_plot(explainer.expected_value, shap_values[0,:], X_test.iloc[0,:], show=False, matplotlib=True)
plt.savefig("shap_force.png")
plt.close()

h2o.shutdown()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM Temurin-11.0.26+4 (build 11.0.26+4, mixed mode)
  Starting server from C:\Users\vegiv\anaconda3\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\vegiv\AppData\Local\Temp\tmp5taeod0i
  JVM stdout: C:\Users\vegiv\AppData\Local\Temp\tmp5taeod0i\h2o_vegiv_started_from_python.out
  JVM stderr: C:\Users\vegiv\AppData\Local\Temp\tmp5taeod0i\h2o_vegiv_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,4 months and 24 days
H2O_cluster_name:,H2O_from_python_vegiv_6gkpz6
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.928 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


C:\Users\vegiv\anaconda3\Lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


  0%|          | 0/50 [00:00<?, ?it/s]

...

C:\Users\vegiv\anaconda3\Lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


Displaying SHAP Force Plot (Local Explanation for first instance)...


C:\Users\vegiv\anaconda3\Lib\site-packages\shap\plots\_force_matplotlib.py:325: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  ax.set_xlim(min_x, max_x)


H2O session _sid_8a3e closed.


C:\Users\vegiv\AppData\Local\Temp\ipykernel_38864\1282517718.py:33: H2ODeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  h2o.shutdown()


Connected SHAP to H2O model. First initialized H2O and loaded the best model from the previous, which was GBM_grid_1_AutoML_3_20250821_115428_model_3.zip on the first execution, and StackedEnsemble_AllModels_1_AutoML_1_20250821_131747.zip on the second.